In [ ]:
%cd /content/drive/My Drive/AA290

import numpy as np
import tensorflow as tf
import data 
import keras_model
import matplotlib.pyplot as plt
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [ ]:
dt=data.Dataset([-200,200],100,8,noisy=True,to_fix=1,n_epochs=1,random_pos=True,p_cycle=0,single_design=True,p_multipath=0,all_multipath=False,min_N=1)

In [ ]:
dt.compute_float()

In [ ]:
np.count_nonzero(np.abs(np.round(dt.x_float[:,:-3])-dt.N)<0.5)/np.prod(dt.N.shape)

In [ ]:
dt.compute_lambda()

In [ ]:
dt.compute_lambda_accuracy()

In [ ]:
x_train,y_train=dt.prepare_data(False,True,False,False,False,False,False,False)
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train1,y_train1=dt.allinone(True,False)
x_train2,y_train2=dt.reg_test()
print(x_train1.shape)
print(y_train2.shape)

In [ ]:
x_trainn=x_train1.copy()
x_trainn[:,:14]=x_train1[:,:14]/(2e2)
x_trainn[:,-45:]=x_train1[:,-45:]/5e2
x_trainn[:,-48:-45]=x_train1[:,-48:-45]/75
x_trainn[:,98:118]=x_train1[:,98:118]/2e2

#x_trainn[:,-46]=x_train1[:,-47]/50
y_trainn=y_train2.copy()
y_trainn[:,:-3]=y_train2[:,:-3]/10
#y_trainn[:,-1]=y_train2[:,-1]/50

In [ ]:
np.std(y_trainn,axis=0)

In [ ]:
xt=x_trainn[:int(0.9*len(x_trainn))]
yt=y_trainn[:int(0.9*len(x_trainn)),:-3]
xv=x_trainn[int(0.9*len(x_trainn)):]
yv=y_trainn[int(0.9*len(x_trainn)):,:-3]

In [ ]:
reg_model=keras_model.FCModel_reg([1000,500,250],0,7,True,0.1)

In [ ]:
def scheduler(epoch, lr):
    if epoch < 200:
      return 1e-3
    elif epoch<400:
      return 1e-4
    elif epoch<800:
      return 1e-5
    return 1e-7
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history=reg_model.fit(xt,yt,batch_size=512,epochs=500,validation_data=(xv,yv),callbacks=[callback])

In [ ]:
pred=reg_model.predict(xv)

In [ ]:
#print(np.mean(np.sum((dt.dx-dt.x_float[:,-3:])**2,axis=1)))
print(np.mean(np.mean((pred-yv)**2,axis=1)))

In [ ]:
np.count_nonzero(np.round(pred*10)==yv*10)/7/len(yv)

In [ ]:
x_train,y_train=dt.all_float(True)
print(x_train.shape)
print(y_train.shape)

In [ ]:
y_train=np.zeros((x_train.shape[0],y_train.shape[1]))
y_train[np.arange(x_train.shape[0]),np.argmax(np.abs(dt.N),axis=1)]=1

In [ ]:
y_train=(dt.N!=0).astype('float')
print(y_train.shape)

In [ ]:
x_train[:,:28]=x_train[:,:28]/100
x_train[:,0:7*4:4]=x_train[:,0:7*4:4]*10
x_train[:,28:]=x_train[:,28:]*10000

In [ ]:
x_trainn=np.zeros_like(x_train)
x_trainn[:,:5*4]=x_train[:,:5*4]/100
x_trainn[:,0:5*4:4]=x_train[:,0:5*4:4]*10
x_trainn[:,5*4:]=x_train[:,5*4:]*10

In [ ]:
mo_model=keras_model.MO_model([],[800,800,800,600,600,600],21,1,(107,),True,False,6,'softmax')

In [ ]:
def scheduler(epoch, lr):
    if epoch < 300:
      return 1e-4
    elif epoch<400:
      return 1e-5
    elif epoch<500:
      return 1e-6
    elif epoch<600:
      return 1e-8
    return 1e-7
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
#history=mo_model.fit(x_trainn,y_train1[:,0,:],batch_size=1024,epochs=500,validation_split=0.05,callbacks=[callback],verbose=2)
history=mo_model.fit(x_trainn,y,batch_size=1024,epochs=600,validation_split=0.1,callbacks=[callback])
#history=mo_model.fit(x_train,[y_train[:,i,:] for i in range(y_train.shape[1])],batch_size=1024,epochs=500,validation_split=0.05,callbacks=[callback],verbose=2)

In [ ]:
name='SO_CONVr_loss.pdf'
plt.figure()
plt.plot(np.arange(len(history.history['loss']))+1,history.history['loss'])
plt.plot(np.arange(len(history.history['loss']))+1,history.history['val_loss'])
plt.legend(['Training loss','Validation loss'])
plt.savefig(name)
plt.show()

In [ ]:
name='SO_CONVr_acc.pdf'
plt.figure()
plt.plot(np.arange(len(history.history['loss']))+1,history.history['acc'])
plt.plot(np.arange(len(history.history['loss']))+1,history.history['val_acc'])
plt.legend(['Training accuracy','Validation accuracy'])
plt.savefig(name)
plt.show()

In [ ]:
y=data.comb_y(dt.N)

In [ ]:
x_train,y_train=dt.prepare_conv(False,False)
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train,y_train=dt.conv_multiepoch(True,False)
print(x_train.shape)
print(y_train.shape)

In [ ]:
conv_model=keras_model.convMO_model([1000,500,250],8,1,(7,4,2,),True)

In [ ]:
def scheduler(epoch, lr):
    if epoch < 200:
      return 1e-4
    elif epoch<400:
      return 1e-5
    elif epoch<800:
      return 1e-6
    return 1e-7
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history=conv_model.fit(x_train,y_train[:,0,:],batch_size=1024,epochs=500,validation_split=0.05,callbacks=[callback])
#history=conv_model.fit(x_train,[y_train[:,i,:] for i in range(y_train.shape[1])],batch_size=2096,epochs=200,validation_split=0.05,callbacks=[callback])

In [ ]:
conv_reg_model=keras_model.conv_reg([500,250],7)

In [ ]:
xt=x_train[:int(0.9*len(x_train))]
yt=y_train[:int(0.9*len(x_train))]
xv=x_train[int(0.9*len(x_train)):]
yv=y_train[int(0.9*len(x_train)):]

In [ ]:
def scheduler(epoch, lr):
    if epoch < 200:
      return 1e-3
    elif epoch<500:
      return 1e-5
    elif epoch<800:
      return 1e-5
    return 1e-7
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
history=conv_reg_model.fit(xt,yt/10,batch_size=512,epochs=300,validation_data=(xv,yv/10),callbacks=[callback])

In [ ]:
pred=conv_reg_model.predict(xv)

In [ ]:
np.count_nonzero(np.round(pred*10)==yv)/7/len(yv)

Random forest test

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf=RandomForestClassifier(n_estimators=200,max_depth=25,n_jobs=-1)
x_rf=x_train[:int(0.8*len(x_train))]
#y_rf=dt.N[:int(0.8*len(x_trainn)),0]
y_rf=y_train[:int(0.8*len(x_train))]
x_val_rf=x_train[int(0.8*len(x_train)):]
#y_val_rf=dt.N[int(0.8*len(x_trainn)):]
y_val_rf=y_train[int(0.8*len(x_train)):]
y_rf=np.argmax(y_rf,axis=1)
y_val_rf=np.argmax(y_val_rf,axis=1)



In [ ]:
y_rf[:10]

In [ ]:
clf.fit(x_rf,y_rf)

In [ ]:
pred=clf.predict(x_val_rf)

In [ ]:
pred[:100]

In [ ]:
np.count_nonzero(y_val_rf)/len(y_val_rf)

In [ ]:
clf.score(x_val_rf,y_val_rf)

In [ ]:
1-np.count_nonzero(pred-y_val_rf)/pred.shape[0]/pred.shape[1]

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
reg=RandomForestRegressor(n_estimators=200,max_depth=25)
x_rf=x_trainn[:int(0.8*len(x_trainn))]
y_rf=y_trainn[:int(0.8*len(x_trainn)),:-3]
x_val_rf=x_trainn[int(0.8*len(x_trainn)):]
y_val_rf=y_trainn[int(0.8*len(x_trainn)):,:-3]
reg.fit(x_rf,y_rf)


In [ ]:
pred=reg.predict(x_val_rf)

In [ ]:
pred[10]*10

In [ ]:
y_val_rf[10]*10

In [ ]:
np.mean(np.sum((pred-y_val_rf)**2,axis=1))

In [ ]:
r=np.round(pred*10)
np.count_nonzero(y_val_rf*10==r)/np.prod(r.shape)

In [ ]:
print(np.mean(np.sum((dt.dx-dt.x_float[:,-3:])**2,axis=1)))
print(np.mean(np.sum(((pred-y_val_rf)*np.array([200,200,50]))**2,axis=1)))